<a href="https://colab.research.google.com/github/shaonirfan/hackathon/blob/main/Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from huggingface_hub import login
login()

In [5]:
# Step 1: Load the Dataset
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Split the dataset into training and validation sets (80/20 split)
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

train_data = train_test_split['train']
val_data = train_test_split['test']

# Inspect the dataset
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(train_data[0])


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

Training samples: 4004
Validation samples: 1002
{'bn': 'এটা কোনো পোস্ট হলো মিয়া আবাল', 'rm': 'eta kono post holo mia abal'}


In [6]:
# Filtering function
def filter_data(example):
    min_len = 3  # Minimum sentence length (in words)
    max_len = 128  # Maximum sentence length (in characters)
    return min_len <= len(example['rm'].split()) <= max_len and \
           min_len <= len(example['bn'].split()) <= max_len

# Apply filtering to both training and validation data
train_data = train_data.filter(filter_data)
val_data = val_data.filter(filter_data)

print(f"Filtered training samples: {len(train_data)}")
print(f"Filtered validation samples: {len(val_data)}")


Filter:   0%|          | 0/4004 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1002 [00:00<?, ? examples/s]

Filtered training samples: 3657
Filtered validation samples: 910


In [8]:
# Step 2: Data Preprocessing
from transformers import AutoTokenizer

# Load a tokenizer for the chosen sequence-to-sequence model
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

# Tokenization function
def tokenize_data(batch):
    tokenized_input = tokenizer(
        batch['rm'],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokenized_output = tokenizer(
        batch['bn'],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    return {
        "input_ids": tokenized_input["input_ids"],
        "attention_mask": tokenized_input["attention_mask"],
        "labels": tokenized_output["input_ids"]
    }

# Apply the tokenization function to the dataset
train_data = train_data.map(tokenize_data, batched=True)
val_data = val_data.map(tokenize_data, batched=True)

# Set the dataset format for PyTorch or TensorFlow
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("Tokenization complete!")


Map:   0%|          | 0/3657 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Tokenization complete!


In [9]:
# Step 3: Load the Pre-trained Model
from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

# Load the pre-trained sequence-to-sequence model
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

# Step 4: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"  # Disables WandB logging
)

# Define the Trainer without tokenizer (updated for compatibility)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

# Step 4: Train the Model
trainer.train()



<ipython-input-9-4c8f0b6e06a4>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,19.804700,13.386039
2,10.403200,6.666318
3,8.380600,5.380552


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=687, training_loss=18.0941189095443, metrics={'train_runtime': 843.7009, 'train_samples_per_second': 13.003, 'train_steps_per_second': 0.814, 'total_flos': 1450228214661120.0, 'train_loss': 18.0941189095443, 'epoch': 3.0})

In [11]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./content/drive/MyDrive/Colab Notebooks/banglish_to_bangla_model")
tokenizer.save_pretrained("./content/drive/MyDrive/Colab Notebooks/banglish_to_bangla_model")

print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!


In [12]:
# Optional: Upload to Hugging Face (requires authentication)
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub("banglish-to-bangla-model")
tokenizer.push_to_hub("banglish-to-bangla-model")

print("Model uploaded to Hugging Face Hub!")


model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Model uploaded to Hugging Face Hub!


In [14]:
# Load the fine-tuned model for inference
from transformers import pipeline

translation_pipeline = pipeline(
    "translation",
    model="./banglish_to_bangla_model",
    tokenizer="./banglish_to_bangla_model"
)

# Example inputs
sample_texts = ["ami banglay gan gai"]

# Generate translations
for text in sample_texts:
    output = translation_pipeline(text)
    print(f"Banglish: {text}")
    print(f"Bangla: {output[0]['translation_text']}")
    print()


Device set to use cuda:0


Banglish: ami banglay gan gai
Bangla: <extra_id_0> 

